In [59]:
import pandas as pd

In [60]:
data = pd.read_csv('train.csv')

In [61]:
data.head()

,Unnamed: 0,DATE,LATITUDE_A,LONGITUDE_A,ELEVATION_A,PRCP_A,SNWD_A,TMAX_A,TMIN_A,TAVG_A,...,LONGITUDE_C,ELEVATION_C,PRCP_C,SNWD_C,TMAX_C,TMIN_C,TAVG_C,LATITUDE,LONGITUDE,TAVG
0,0,01-11-1979,63.88,77.52,128,NaN,0.0,17.6,NaN,12.96,...,78.93,64,0.0,0.00,18.7,NaN,NaN,63.933,78.733,9.3
1,1,13-11-1979,63.88,77.52,128,NaN,0.0,8.8,NaN,0.00,...,78.93,64,0.0,0.00,11.0,-3.8,2.04,63.933,78.733,2.7
2,2,05-02-1980,63.88,77.52,128,NaN,0.0,-9.9,NaN,-17.76,...,78.93,64,0.0,0.00,NaN,NaN,-16.56,63.933,78.733,-16.0
3,3,23-12-1979,63.88,77.52,128,0.0,0.0,NaN,-4.30,-4.44,...,78.93,64,0.0,0.00,0.0,NaN,-2.64,63.933,78.733,-3.2
4,4,07-06-1977,63.88,77.52,128,NaN,0.0,39.6,15.48,NaN,...,78.93,64,NaN,3.82,38.5,17.1,33.00,63.933,78.733,27.3


In [62]:
cty = {'A': {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0}, 'B': {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0}, 'C': {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0}}
# cty = {'A': 0, 'B': 0, 'C': 0}
# fld = {'TMAX': 0, 'TMIN': 0, 'TAVG': 0, 'PRCP': 0, 'SNWD': 0}
# cty = {i: fld for i, x in cty.items()}
# print(cty)
for x, dat in enumerate(data.itertuples()):
    for key, val in cty.items():
        for ind, tvl in val.items():
            rem = list(val.keys())
            rem.remove(ind)
            if pd.isna(getattr(dat, f'{ind}_{key}')):
                v1, v2 = getattr(dat, f'{rem[0]}_{key}'), getattr(dat, f'{rem[1]}_{key}')
                if not pd.isna(v1) and not pd.isna(v2) and ind != 'PRCP' and ind != 'SNWD':
                    data.loc[x, f'{ind}_{key}'] = ((2 * v2) - v1) if ind in ['TMAX', 'TMIN'] else ((v1 + v2) / 2)
                    cty[key][ind] += 1
                else:
                    rec = list(cty.keys())
                    rec.remove(key)
                    for ciy in rec:
                        if not pd.isna(getattr(dat, f'{ind}_{ciy}')):
                            data.loc[x, f'{ind}_{key}'] = float(getattr(dat, f'{ind}_{ciy}'))
                            cty[key][ind] += 1
                            break
        #%%
print(cty)

{'A': {'TMAX': 245, 'TMIN': 523, 'TAVG': 70, 'PRCP': 561, 'SNWD': 0}, 'B': {'TMAX': 0, 'TMIN': 0, 'TAVG': 70, 'PRCP': 0, 'SNWD': 0}, 'C': {'TMAX': 239, 'TMIN': 471, 'TAVG': 129, 'PRCP': 207, 'SNWD': 0}}


In [63]:
data.isnull().sum()

Unnamed: 0     0
DATE           0
LATITUDE_A     0
LONGITUDE_A    0
ELEVATION_A    0
PRCP_A         0
SNWD_A         0
TMAX_A         0
TMIN_A         0
TAVG_A         0
LATITUDE_B     0
LONGITUDE_B    0
ELEVATION_B    0
PRCP_B         0
SNWD_B         0
TMAX_B         0
TMIN_B         0
TAVG_B         0
LATITUDE_C     0
LONGITUDE_C    0
ELEVATION_C    0
PRCP_C         0
SNWD_C         0
TMAX_C         0
TMIN_C         0
TAVG_C         0
LATITUDE       0
LONGITUDE      0
TAVG           0
dtype: int64

In [64]:
data = data.drop(['Unnamed: 0'], axis=1)

In [65]:
from sklearn.model_selection import train_test_split

X, Y = data.drop(['TAVG', 'DATE'], axis=1), data['TAVG']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
train = X_train.join(y_train)

In [66]:
from sklearn.ensemble import RandomForestRegressor
forest=RandomForestRegressor()
forest.fit(X_train, y_train)


RandomForestRegressor()

In [67]:
forest.score(X_test, y_test)

0.972610510480686

In [68]:
test = pd.read_csv('test.csv')

In [69]:
data.columns

Index(['DATE', 'LATITUDE_A', 'LONGITUDE_A', 'ELEVATION_A', 'PRCP_A', 'SNWD_A',
       'TMAX_A', 'TMIN_A', 'TAVG_A', 'LATITUDE_B', 'LONGITUDE_B',
       'ELEVATION_B', 'PRCP_B', 'SNWD_B', 'TMAX_B', 'TMIN_B', 'TAVG_B',
       'LATITUDE_C', 'LONGITUDE_C', 'ELEVATION_C', 'PRCP_C', 'SNWD_C',
       'TMAX_C', 'TMIN_C', 'TAVG_C', 'LATITUDE', 'LONGITUDE', 'TAVG'],
      dtype='object')

In [70]:
test.columns

Index(['INDEX', 'DATE', 'LATITUDE_A', 'LONGITUDE_A', 'ELEVATION_A', 'PRCP_A',
       'SNWD_A', 'TMAX_A', 'TMIN_A', 'TAVG_A', 'LATITUDE_B', 'LONGITUDE_B',
       'ELEVATION_B', 'PRCP_B', 'SNWD_B', 'TMAX_B', 'TMIN_B', 'TAVG_B',
       'LATITUDE_C', 'LONGITUDE_C', 'ELEVATION_C', 'PRCP_C', 'SNWD_C',
       'TMAX_C', 'TMIN_C', 'TAVG_C', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [71]:
test = test.drop(['INDEX'], axis=1)

In [72]:
for x, dat in enumerate(test.itertuples()):
    for key, val in cty.items():
        for ind, tvl in val.items():
            rem = list(val.keys())
            rem.remove(ind)
            if pd.isna(getattr(dat, f'{ind}_{key}')):
                v1, v2 = getattr(dat, f'{rem[0]}_{key}'), getattr(dat, f'{rem[1]}_{key}')
                if not pd.isna(v1) and not pd.isna(v2) and ind != 'PRCP' and ind != 'SNWD':
                    test.loc[x, f'{ind}_{key}'] = ((2 * v2) - v1) if ind in ['TMAX', 'TMIN'] else ((v1 + v2) / 2)
                else:
                    rec = list(cty.keys())
                    rec.remove(key)
                    for ciy in rec:
                        if not pd.isna(getattr(dat, f'{ind}_{ciy}')):
                            test.loc[x, f'{ind}_{key}'] = float(getattr(dat, f'{ind}_{ciy}'))
                            break

In [73]:
test.isnull().sum()

DATE           0
LATITUDE_A     0
LONGITUDE_A    0
ELEVATION_A    0
PRCP_A         0
SNWD_A         0
TMAX_A         0
TMIN_A         0
TAVG_A         0
LATITUDE_B     0
LONGITUDE_B    0
ELEVATION_B    0
PRCP_B         0
SNWD_B         0
TMAX_B         0
TMIN_B         0
TAVG_B         0
LATITUDE_C     0
LONGITUDE_C    0
ELEVATION_C    0
PRCP_C         0
SNWD_C         0
TMAX_C         0
TMIN_C         0
TAVG_C         0
LATITUDE       0
LONGITUDE      0
dtype: int64

In [74]:
X_test = test.drop(['DATE'], axis=1)
y_pred = forest.predict(X_test) 

In [75]:
y_pred

array([ 9.2300e-01,  1.0550e+01,  1.2888e+01,  8.8420e+00,  1.8534e+01,
        3.6100e-01,  1.9125e+01,  7.7290e+00,  8.6340e+00,  2.0272e+01,
        6.2650e+00,  7.5900e+00,  1.1278e+01,  2.4213e+01,  1.9536e+01,
       -1.8490e+00,  1.9476e+01,  2.6257e+01,  8.2000e-02,  2.4950e+00,
        2.4706e+01, -3.7660e+00,  2.3900e-01, -2.5120e+00,  1.9135e+01,
       -5.9970e+00,  2.5500e-01,  2.4856e+01,  1.1243e+01,  4.5830e+00,
        2.3967e+01,  2.0850e+01,  1.9778e+01,  2.2250e+01, -2.3200e-01,
        1.5873e+01,  2.1357e+01,  2.1333e+01, -9.8900e+00, -1.0000e+01,
        2.3729e+01, -5.7750e+00, -6.7930e+00,  2.1238e+01,  1.2119e+01,
        2.7167e+01,  1.8954e+01, -1.1782e+01, -1.5324e+01, -5.6610e+00,
       -1.9900e-01,  2.4136e+01,  2.3537e+01, -3.1780e+00,  5.3080e+00,
        1.5043e+01,  9.3900e-01, -8.9900e+00,  2.0665e+01, -1.3287e+01,
        3.3210e+00,  1.2309e+01,  8.7400e-01,  2.0078e+01,  1.4997e+01,
        8.0000e-03,  1.9488e+01, -4.4000e-01,  6.6120e+00,  1.16

In [76]:
forest.score(X_test, y_pred)

1.0

In [81]:
df = pd.DataFrame({'TAVG':y_pred}, index=test.index)

In [82]:
df.head()

,TAVG
0,0.923
1,10.550
2,12.888
3,8.842
4,18.534


In [84]:
df.to_csv('sample_submission.csv')